In [1]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import json
from fake_useragent import UserAgent

In [2]:
ua = UserAgent()
headers = {"user-agent": ua.chrome}

In [3]:
import pandas as pd
import numpy as np

### Loading list of toys

In [16]:
with open('all_items.txt') as f:
    contents = f.read().splitlines()
contents

['vehicle toys',
 'sport',
 'military toys',
 'race cars',
 'outer space toys',
 'depots',
 'machines',
 'doll-humanoid',
 'action figures',
 'gi joe action figure',
 'spiderman',
 'guitar',
 'castle tent',
 'microscope',
 'monster trucks',
 'bug collection set',
 'barn',
 'grill',
 'dinosaur toy',
 'volcano creator',
 'pokemon cards',
 'space station toys',
 'basketball hoop',
 'gears',
 'train set',
 'lincoln logs',
 'police station toy',
 'airport toy',
 'police officer gear',
 'dragonballz',
 'superhero costumes',
 'helicopter',
 'racetrack',
 'remote control car',
 'football',
 'hockey stick',
 'sword toy',
 'miniature guns',
 'truck set',
 'remote control helicopter',
 'sully costume',
 'woody valentine',
 'spiderman costume',
 'batman costume',
 'ninja costume',
 'wwe action figure',
 'action figure',
 'toy gun',
 'construction games',
 'war toys',
 'toy vehicles',
 'educational material',
 'nerf guns',
 'construction toys',
 'car toys',
 'video games',
 'building blocks',
 'din

stereo_books = pd.read_csv('../data/stereotyped_books.csv', delimiter =',')
stereo_books

with open('book_list.txt') as f:
    books = f.read().splitlines()

books

## Loading pre-documented gender stereotyped toys

In [9]:
stereo_toys = pd.read_csv('~/Desktop/485/Data/predoc_stereotyped_items.csv', delimiter =',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


In [ ]:
#stereo_toys.dropna()

stereo_toys['BOY'] = stereo_toys['BOY'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys['GIRL'] = stereo_toys['GIRL'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys['NEUTRAL'] = stereo_toys['NEUTRAL'].apply(lambda x: x.lower() if isinstance(x, str) else x)
stereo_toys

stereo_toys.to_csv('../data/stereotyped_items.csv', index=False)

### Some stat on the dataset

In [10]:
boy_toys = stereo_toys['BOY'].unique().tolist()
len(boy_toys)

71

In [11]:
girl_toys = stereo_toys['GIRL'].unique().tolist()
len(girl_toys)

64

In [12]:
neutral_toys = stereo_toys['NEUTRAL'].unique().tolist()
len(neutral_toys)

33

In [13]:
all_items = boy_toys + girl_toys + neutral_toys
len(all_items)

168

In [20]:
all_items = [x for x in all_items if isinstance(x, str)]

## Preparing the data for query

textfile = open("all_items.txt", "w")
for element in all_items:
    textfile.write(element + "\n")
textfile.close()

In [21]:
#contents

### Adding "for"

In [23]:
new_content = [x.strip() for x in all_items]
new_content = [x.replace(' for','') for x in new_content]
new_content = [x.lower() for x in new_content]
new_content

['vehicle toys',
 'sport',
 'military toys',
 'race cars',
 'outer space toys',
 'depots',
 'machines',
 'doll-humanoid',
 'action figures',
 'gi joe action figure',
 'spiderman',
 'guitar',
 'castle tent',
 'microscope',
 'monster trucks',
 'bug collection set',
 'barn',
 'grill',
 'dinosaur toy',
 'volcano creator',
 'pokemon cards',
 'space station toys',
 'basketball hoop',
 'gears',
 'train set',
 'lincoln logs',
 'police station toy',
 'airport toy',
 'police officer gear',
 'dragonballz',
 'superhero costumes',
 'helicopter',
 'racetrack',
 'remote control car',
 'football',
 'hockey stick',
 'sword toy',
 'miniature guns',
 'truck set',
 'remote control helicopter',
 'sully costume',
 'woody valentine',
 'spiderman costume',
 'batman costume',
 'ninja costume',
 'wwe action figure',
 'action figure',
 'toy gun',
 'construction games',
 'war toys',
 'toy vehicles',
 'educational material',
 'nerf guns',
 'construction toys',
 'car toys',
 'video games',
 'building blocks',
 'din

In [24]:
search_terms = [x+' for' for x in new_content]
search_terms

['vehicle toys for',
 'sport for',
 'military toys for',
 'race cars for',
 'outer space toys for',
 'depots for',
 'machines for',
 'doll-humanoid for',
 'action figures for',
 'gi joe action figure for',
 'spiderman for',
 'guitar for',
 'castle tent for',
 'microscope for',
 'monster trucks for',
 'bug collection set for',
 'barn for',
 'grill for',
 'dinosaur toy for',
 'volcano creator for',
 'pokemon cards for',
 'space station toys for',
 'basketball hoop for',
 'gears for',
 'train set for',
 'lincoln logs for',
 'police station toy for',
 'airport toy for',
 'police officer gear for',
 'dragonballz for',
 'superhero costumes for',
 'helicopter for',
 'racetrack for',
 'remote control car for',
 'football for',
 'hockey stick for',
 'sword toy for',
 'miniature guns for',
 'truck set for',
 'remote control helicopter for',
 'sully costume for',
 'woody valentine for',
 'spiderman costume for',
 'batman costume for',
 'ninja costume for',
 'wwe action figure for',
 'action figur

search_books = [x+' for' for x in books]
search_books

## Search Engines and E-Com Sites

### Amazon

In [25]:
def amazon_scrape(query):
    url = f"https://completion.amazon.com/api/2017/suggestions?session-id=131-6901588-5783061&customer-id=A373R49950VTB6&request-id=72HVV86S4JC3AK898B3X&page-type=Gateway&lop=en_gb&site-variant=desktop&client-info=amazon-search-ui&mid=ATVPDKIKX0DER&alias=aps&ks=undefined&prefix={query}&event=onFocusWithSearchTerm&limit=11&b2b=0&fresh=0&fb=1&suggestion-type=KEYWORD&suggestion-type=WIDGET&_=1637596795610"
    response = requests.get(url, headers=headers, verify=False).json()
    return response

### All together

In [26]:
def amazon_auto(item):
    res = amazon_scrape(item)
    suggestions = res['suggestions']
    results = []
    for s in suggestions:
        results.append(s['value'])
    return results

## Run queries

In [27]:
import pandas as pd
columns = ['platform', 'query', 'suggestions']
df = pd.DataFrame(columns = columns)
df

,platform,query,suggestions


In [37]:
def search(platform):
    if platform == 'Amazon_UK':
        return amazon_auto(item)

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
platforms = ['Amazon_UK']
data = []
for item in contents:
    for platform in platforms:
        result = search(platform) 
        values = [platform, item, result]
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        #print(a_dictionary)
        time.sleep(1.5)
        data.append(a_dictionary)

In [36]:
data

[{'platform': 'Amazon_UK', 'query': 'vehicle toys', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'sport', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'military toys', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'race cars', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'outer space toys', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'depots', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'machines', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'doll-humanoid', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'action figures', 'suggestions': None},
 {'platform': 'Amazon_UK',
  'query': 'gi joe action figure',
  'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'spiderman', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'guitar', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'castle tent', 'suggestions': None},
 {'platform': 'Amazon_UK', 'query': 'microscope'

In [31]:
df = df.append(data, True)
df

,platform,query,suggestions
0,Amazon,vehicle toys,"[vehicle toys, vehicle toys for boys 3-5 years..."
1,Amazon,sport,"[sports bras for women, sports bra, sports gif..."
2,Amazon,military toys,"[military toys, military toys for boys age 8-1..."
3,Amazon,race cars,"[race cars, race cars for kids ages 3-5, race ..."
4,Amazon,outer space toys,"[outer space toys, outer space toys for kids 5..."
...,...,...,...
128,Amazon,playhouses,"[playhouses, playhouses for kids outdoor, play..."
129,Amazon,princess sword,"[princess sword, princess sword and shield, pr..."
130,Amazon,doctors kit,"[doctors kit, doctors kit for 3 year old, doct..."
131,Amazon,skin care kit,"[skin care kit, skin care kit for teen girls, ..."


In [32]:
df.to_csv('az_uk_query_suggestions.csv', index = False)